In [5]:
# python script of M4_weekly and DeepAR

# standard imports
import numpy as np
import pandas as pd

# json
import json

# gluon data
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator


# model imports
# from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint

# seeds
mx.random.seed(42)
np.random.seed(42)

if __name__ == "__main__":

    dataset = get_dataset("m4_monthly", regenerate=False)

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=200,      # default: 100
        num_batches_per_epoch=100,      #default: 50
        learning_rate=1e-3,
        # hybridize=False,
    )

    # cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        # context_length=168,
        # cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )

    pprint(agg_metrics)


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_monthly.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 13463
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:07<00:00, 13.14it/s, avg_epoch_loss=7.4]
INFO:root:Epoch[0] Elapsed time 7.632 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.403031
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:09<00:00, 10.56it/s, avg_epoch_loss=7.63]
INFO:root:Epoch[1] Elapsed time 9.479 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.633294
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:10<00:00,  9.56it/s, avg_epoch_loss=6.98]
INFO:root:Epoch[2] Elapsed time 10.481 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.976654
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:09<00:00, 10.11it/s, avg_epoch_loss=6.75]
INFO:root:Epoch[3] Elapsed time 9.916 se

INFO:root:Epoch[36] Learning rate is 0.0005
100%|██████████| 100/100 [00:08<00:00, 11.72it/s, avg_epoch_loss=6.46]
INFO:root:Epoch[36] Elapsed time 8.538 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=6.456984
INFO:root:Epoch[37] Learning rate is 0.0005
100%|██████████| 100/100 [00:08<00:00, 12.37it/s, avg_epoch_loss=6.08]
INFO:root:Epoch[37] Elapsed time 8.093 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=6.082327
INFO:root:Epoch[38] Learning rate is 0.0005
100%|██████████| 100/100 [00:08<00:00, 11.83it/s, avg_epoch_loss=6.79]
INFO:root:Epoch[38] Elapsed time 8.459 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=6.794083
INFO:root:Epoch[39] Learning rate is 0.0005
100%|██████████| 100/100 [00:08<00:00, 11.29it/s, avg_epoch_loss=7.37]
INFO:root:Epoch[39] Elapsed time 8.865 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=7.365226
INFO:root:Epoch[40] Learning rate is 0.0005
100%|██████████| 100/100 [00:08<00:00, 11.57it/s, avg_epoch_loss=6.0

100%|██████████| 100/100 [00:09<00:00, 10.71it/s, avg_epoch_loss=6.99]
INFO:root:Epoch[72] Elapsed time 9.343 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=6.988359
INFO:root:Loading parameters from best epoch (52)
INFO:root:Epoch[73] Learning rate is 6.25e-05
100%|██████████| 100/100 [00:08<00:00, 11.34it/s, avg_epoch_loss=6.08]
INFO:root:Epoch[73] Elapsed time 8.828 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=6.084116
INFO:root:Epoch[74] Learning rate is 6.25e-05
100%|██████████| 100/100 [00:09<00:00, 10.83it/s, avg_epoch_loss=7.6]
INFO:root:Epoch[74] Elapsed time 9.245 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=7.603065
INFO:root:Epoch[75] Learning rate is 6.25e-05
100%|██████████| 100/100 [00:09<00:00, 11.10it/s, avg_epoch_loss=7.11]
INFO:root:Epoch[75] Elapsed time 9.022 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=7.110630
INFO:root:Epoch[76] Learning rate is 6.25e-05
100%|██████████| 100/100 [00:08<00:00, 11.14it/s, avg_e

INFO:root:Epoch[108] Elapsed time 9.672 seconds
INFO:root:Epoch[108] Evaluation metric 'epoch_loss'=7.148403
INFO:root:Epoch[109] Learning rate is 5e-05
100%|██████████| 100/100 [00:08<00:00, 11.73it/s, avg_epoch_loss=5.9]
INFO:root:Epoch[109] Elapsed time 8.532 seconds
INFO:root:Epoch[109] Evaluation metric 'epoch_loss'=5.899435
INFO:root:Epoch[110] Learning rate is 5e-05
100%|██████████| 100/100 [00:08<00:00, 11.14it/s, avg_epoch_loss=6.63]
INFO:root:Epoch[110] Elapsed time 8.994 seconds
INFO:root:Epoch[110] Evaluation metric 'epoch_loss'=6.632881
INFO:root:Epoch[111] Learning rate is 5e-05
100%|██████████| 100/100 [00:08<00:00, 11.56it/s, avg_epoch_loss=6.18]
INFO:root:Epoch[111] Elapsed time 8.659 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=6.182851
INFO:root:Epoch[112] Learning rate is 5e-05
100%|██████████| 100/100 [00:09<00:00, 11.04it/s, avg_epoch_loss=5.82]
INFO:root:Epoch[112] Elapsed time 9.068 seconds
INFO:root:Epoch[112] Evaluation metric 'epoch_loss'=5.824

KeyboardInterrupt: 